# Design Fair Demo

In [7]:
!python ..\data_processing\data_processing\process_data.py "C:\Users\meria\Pictures\Camera Roll" ..\data C:\Users\meria\Documents\GazePoint\result

Traceback (most recent call last):
  File "c:\Users\meria\source\repos\emotion-watchers\data_processing\data_processing\process_data.py", line 72, in <module>
    process_data(
  File "c:\Users\meria\source\repos\emotion-watchers\data_processing\data_processing\process_data.py", line 21, in process_data
    face.process_data(video_dir, output_path, binary_face, skip_get_frames, skip_crop_images, log)
  File "c:\users\meria\source\repos\emotion-watchers\data_processing\data_processing\face\process_data.py", line 220, in process_data
    separate_images(image_dirs, output_path, binary)
  File "c:\users\meria\source\repos\emotion-watchers\data_processing\data_processing\face\process_data.py", line 98, in separate_images
    matched_emotion = emotions[match["emotion"]]
KeyError: 'Pro'
